In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
import tensorflow as tf

import os
import matplotlib as mpl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id = '1px3LZ7Twk5YuPio48YpG4Iy3Li9ag0bC'

In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('telemetry.csv')  
df = pd.read_csv('telemetry.csv', parse_dates=['datetime'], index_col='datetime')
df.head()

In [0]:
result_pressure=seasonal_decompose(df.pressure)
result_volt=seasonal_decompose(df.volt)
result_vibration=seasonal_decompose(df.vibration)
result_rotate=seasonal_decompose(df.rotate)
datetime=np.array(df.index)
pressure=np.array(result_pressure.trend)
volt=np.array(result_volt.trend)
vibration=np.array(result_vibration.trend)
rotate=np.array(result_rotate.trend)
yval=np.array(df.yval)
df=pd.DataFrame((datetime,pressure,volt,vibration,rotate,yval),index=('datetime','pressure','volt','vibration','rotate','yval')).T.dropna()

In [0]:
df.head(8737)

In [0]:
hoursto = np.zeros(8737)
hourssince = np.zeros(8737)
l = len(df['yval'])
temp1 = None
temp2 = None
for i in range(8748, 11, -1):
  if df['yval'][i] == 1:
    temp1 = 1
    hoursto[(i-12)] = 0
    continue
  if temp1 == 1:
    hoursto[(i-12)] = hoursto[(i-12)+1] + 1
  else:
    hoursto[(i-12)] = None
  if df['yval'][8748-(i-12)] == 1:
    temp2 = 1
    hourssince[8736-(i-12)] = 0
    continue
  if temp2 == 1:
    hourssince[8736-(i-12)] = hourssince[8736-(i-12)-1] + 1
  else:
    hourssince[8736-(i-12)] = None
df['hoursto'] = hoursto
df['hourssince'] = hourssince

In [0]:
df.head(8761)

In [0]:
features_considered = ['volt', 'rotate', 'pressure', 'vibration', 'hourssince', 'hoursto']
features = df[features_considered]
index = ['datetime', 'machineID']
features.index = df['datetime']
features.head()

In [0]:
# features.plot(subplots=True)

In [0]:
TRAIN_SPLIT = 7157
dataset = features.values

In [0]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

In [0]:
def plot_train_history(history, title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  plt.figure()

  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()

  plt.show()

In [0]:
def create_time_steps(length):
  return list(range(-length, 0))

In [0]:
def show_plot(plot_data, delta, title):
  labels = ['History', 'True Future', 'Model Prediction']
  marker = ['.-', 'rx', 'go']
  time_steps = create_time_steps(plot_data[0].shape[0])
  if delta:
    future = delta
  else:
    future = 0

  plt.title(title)
  for i, x in enumerate(plot_data):
    if i:
      plt.plot(future, plot_data[i], marker[i], markersize=10,
               label=labels[i])
    else:
      plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
  plt.legend()
  plt.xlim([time_steps[0], (future+5)*2])
  plt.xlabel('Time-Step')
  return plt

In [0]:
past_history = 744
future_target = 0
STEP = 1
print(len(dataset))

In [0]:
x_train_single, y_train_single = multivariate_data(dataset[:, :5], dataset[:, 5], 0,
                                                 TRAIN_SPLIT-future_target+1, past_history,
                                                 future_target, STEP, True)
x_val_single, y_val_single = multivariate_data(dataset[:, :5], dataset[:, 5],
                                             TRAIN_SPLIT, None, past_history,
                                             future_target, STEP, True)
y_train_single.shape

In [0]:
BUFFER_SIZE = 1000
BATCH_SIZE = 2
train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE)

In [0]:
single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(256, return_sequences=True, input_shape=x_train_single.shape[-2:]))
single_step_model.add(tf.keras.layers.LSTM(256))
single_step_model.add(tf.keras.layers.Flatten())
# single_step_model.add(tf.keras.layers.Conv1D(filters=200, kernel_size=5, strides=1, padding="causal", activation="relu"))
single_step_model.add(tf.keras.layers.Dense(180))
single_step_model.add(tf.keras.layers.Dense(1, activation='relu'))

single_step_model.compile(optimizer='sgd', loss='mae')
print(single_step_model.summary())

In [0]:
for x, y in val_data_single.take(1):
  print (single_step_model.predict(x).shape)

In [0]:
EVALUATION_INTERVAL = 300
EPOCHS = 2

In [0]:
single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_single, validation_steps=100)

In [0]:
plot_train_history(single_step_history, 'single-Step Training and validation loss')

In [0]:
y1 = None
for x, y in val_data_single.take(5):
  if y1 != None:
    y0 = single_step_model.predict(x)[0]
    print(y0)
    print(y)
    print(x[:, :, 4])
#     z = [y1.numpy(), y[BUFFER_SIZE-1].numpy(), y0]
#     show_plot(z, 0, 'Prediction')
  y1 = y